In [7]:
from qutip import *
from pylab import *
from scipy import integrate
import time
from numpy import *
import multiprocessing as mp
qutip.settings.num_cpus = 12

In [8]:
def mean_number_photons(delta,input_list,times,psi0):

    """
    This function handles the time evolution and returns the mean number 
    of photons for each value of detuning.
    """
    
    Nat = input_list[0]
    N = input_list[1] 
    kappa = input_list[2]
    g = input_list[3]
    EM = input_list[4]
    Oc = input_list[5]
    norm = input_list[6]
    
    G31 = input_list[7]
    G32 = input_list[8]
    D1 = input_list[9]
    D2 = input_list[10]
    
    #field operator
    op_list = []
    for m in range(Nat+1):
        op_list.append(qeye(3))
    op_list[0] = destroy(N)
    a = tensor(op_list) #muda conforme o número de átoomos
    
    #atomic operators
    S11_list = []
    S22_list = []
    S33_list = []
    S13_list = []
    S23_list = []
    
    for n in range(Nat):
        op_list = []
        for m in range(Nat+1):
            op_list.append(qeye(3))

        op_list[0] = qeye(N)
        
        op_list[n+1] = basis(3,0)*basis(3,0).dag()
        S11_list.append(tensor(op_list))

        op_list[n+1] = basis(3,1)*basis(3,1).dag()
        S22_list.append(tensor(op_list))

        op_list[n+1] = basis(3,2)*basis(3,2).dag()
        S33_list.append(tensor(op_list))
        
        op_list[n+1] = basis(3,0)*basis(3,2).dag()
        S13_list.append(tensor(op_list))
        
        op_list[n+1] = basis(3,1)*basis(3,2).dag()
        S23_list.append(tensor(op_list))        
       
    #total atomic operators
    S11 = 0
    S22 = 0
    S33 = 0
    S13 = 0
    S23 = 0
    
    for n in range(Nat):
        S11 = S11 + S11_list[n]
        S22 = S22 + S22_list[n]
        S33 = S33 + S33_list[n]
        S13 = S13 + S13_list[n]
        S23 = S23 + S23_list[n]

    #colapse Operators
    C = sqrt(kappa)*a   #cavity  
    C31_list = []
    C32_list = []
    
    for n in range(Nat):
        C31_list.append(sqrt(G31)*S13_list[n])
        C32_list.append(sqrt(G32)*S23_list[n])
    
    Clist = [C] + C31_list + C32_list
    
    #Hamiltonian    
    #Hp = (EM/2)*(a + a.dag())
    #Hi = g*(a*S13.dag() + a.dag()*S13)
    #Hc = Oc*(S23 + S23.dag())
    #H0 = D1*S33 + (D1-D2)*S22 + delta*S11 - delta*(a.dag()*a)

    # H_total     
    #Ht = H0 + Hp + Hi + Hc
    Ht = D1*S33 + (D1-D2)*S22 + delta*S11 - delta*a.dag()*a + (g*a*S13.dag() + Oc*S23.dag() + EM*a + g*a.dag()*S13 + Oc*S23 + EM*a.dag())
        

    
    # monte carlo solver
    #data = mcsolve(Ht, psi0, times, Clist, [a.dag() * a], progress_bar=False,ntraj = 24)
    #nm_at = data.expect[0]/norm
    #nm_a = nm_at[-1]
    
#steady state method (demands a high amount of memory as we increase the system's dimension)
    rho_a = steadystate(Ht, Clist, method='eigen', use_rcm=True, maxiter=1000, tol=1e-12)
    nm_a = expect(a.dag()*a, rho_a)/norm  # expectation value of nm -> normalized

    return nm_a

def width_full(NMa,delta_list,Oc):
    
    #maximum value of expected mean number of photons
    high_a = max(NMa) 
    
    #width calculation    
    for i in range(len(delta_list)):
        if NMa[i]>high_a/2:
            width_a = 2*(delta_list[i])
            
    #saving data
    Transmission_data = np.vstack((delta_list,NMa))
    file_data_store('Oc=%.2f.dat'%Oc, Transmission_data.T, numtype="real", numformat="decimal", sep= " ") 
        
    return width_a

def width_break(input_list,delta_list,times,psi0):
    
    for delta in delta_list:
            
            NMa = mean_number_photons(delta, input_list, times, psi0)
            
            if (delta == 0):
                high_a = NMa
    
            if (NMa > high_a/2):
                width_a = 2*(delta)
            else:
                wid = width_a
                break
    
    return wid

In [9]:
if __name__ == '__main__':   
    
    ini = time.time() #starts counting time
    print(time.ctime()) #print initial time

    nsteps1 = 600 #number of steps in the parameter list
    Oc_list = linspace(10**(-3), 5.0, nsteps1) #Rabi frequency list 
    index_j = range(nsteps1) #length of the parameter list

    nsteps = 1200 #number of steps for the detuning list
    delta_list = linspace(0, 2.5, nsteps) #detuning list
    index_i = range(nsteps) #length of the detuning list

    Nat = 2    
    N = 4 #Fock space dimension
    kappa = 1.0 #cavity mode dissipation rate (this is why the parameters should be passed as arguments to the function func1)
    g = (5.0/sqrt(Nat))*kappa #atom-field coupling strenght
    EM = sqrt(0.1)*kappa #pump field strength
    norm = (1.0*kappa)*(EM/kappa)**2 #normalization

    #decay rates
    G31 = 0.5*kappa  #atom
    G32 = 0.5*kappa  #atom

    #detunings
    D1 = 0.0
    D2 = 0.0
        
    times = np.linspace(0.0, 150.0, 250) #evolution time list

    aux_list = []
    for m in range(Nat+1):
        aux_list.append(basis(3,0))
    aux_list[0] = basis(N,0)
    psi0 = tensor(aux_list)
    
    #create empty lists to store the calculated FWHMs
    Wa = []
    
    for j in index_j:
        
        #transmission curve
        Oc = Oc_list[j]
        
        input_list = [Nat,N,kappa,g,EM,Oc,norm,G31,G32,D1,D2]
        
        NMa = [mean_number_photons(delta, input_list, times, psi0) for delta in delta_list]
        width_a = width_full(NMa,delta_list,Oc)
        Wa.append(width_a)
        
        # ##or       
        # width_a = width_break(input_list, delta_list, times, psi0)
        # Wa.append(width_a)
        
            
        #progress tracking
        fin = time.time() 
        print("%f %f %.2f" % (Oc,width_a,(fin-ini)/3600))     
        
        
    output_data = np.vstack((Oc_list, Wa))   # join Oc and expt data
    file_data_store('test.dat', output_data.T, numtype="real", numformat="decimal", sep= " ") 
    

Fri Feb 11 11:36:55 2022
0.001000 1.017515 0.07
0.009346 1.013344 0.14
0.017691 1.005004 0.21
0.026037 0.988324 0.27
0.034382 0.959133 0.34
0.042728 0.921601 0.41
0.051073 0.871560 0.47
0.059419 0.813178 0.54
0.067765 0.750626 0.60
0.076110 0.688073 0.67
0.084456 0.629691 0.74
0.092801 0.575480 0.80
0.101147 0.533778 0.87
0.109492 0.496247 0.94
0.117838 0.467056 1.00
0.126184 0.442035 1.07
0.134529 0.417014 1.13
0.142875 0.396163 1.20
0.151220 0.379483 1.27
0.159566 0.366972 1.33
0.167912 0.350292 1.40
0.176257 0.337781 1.47
0.184603 0.329441 1.53
0.192948 0.316931 1.60
0.201294 0.308590 1.67
0.209639 0.300250 1.73
0.217985 0.291910 1.80
0.226331 0.283570 1.87
0.234676 0.275229 1.93
0.243022 0.266889 2.00
0.251367 0.262719 2.06
0.259713 0.254379 2.13
0.268058 0.250209 2.20
0.276404 0.246038 2.26
0.284750 0.241868 2.33
0.293095 0.237698 2.40
0.301441 0.233528 2.46
0.309786 0.229358 2.53
0.318132 0.225188 2.59
0.326477 0.221018 2.66
0.334823 0.221018 2.73
0.343169 0.216847 2.79
0.351514 

2.896915 0.375313 23.01
2.905260 0.375313 23.08
2.913606 0.375313 23.14
2.921952 0.375313 23.21
2.930297 0.375313 23.27
2.938643 0.379483 23.34
2.946988 0.379483 23.40
2.955334 0.379483 23.47
2.963679 0.379483 23.53
2.972025 0.379483 23.60
2.980371 0.383653 23.66
2.988716 0.383653 23.73
2.997062 0.383653 23.79
3.005407 0.383653 23.86
3.013753 0.383653 23.92
3.022098 0.387823 23.99
3.030444 0.387823 24.05
3.038790 0.387823 24.12
3.047135 0.387823 24.18
3.055481 0.387823 24.24
3.063826 0.387823 24.31
3.072172 0.391993 24.38
3.080518 0.391993 24.44
3.088863 0.391993 24.51
3.097209 0.391993 24.57
3.105554 0.391993 24.64
3.113900 0.396163 24.70
3.122245 0.396163 24.77
3.130591 0.396163 24.83
3.138937 0.396163 24.90
3.147282 0.396163 24.96
3.155628 0.400334 25.03
3.163973 0.400334 25.09
3.172319 0.400334 25.16
3.180664 0.400334 25.23
3.189010 0.400334 25.29
3.197356 0.404504 25.36
3.205701 0.404504 25.42
3.214047 0.404504 25.49
3.222392 0.404504 25.55
3.230738 0.404504 25.62
3.239083 0.40867